In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [9]:
%time SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : alpha*c})

CPU times: user 5.71 s, sys: 4.59 ms, total: 5.72 s
Wall time: 5.73 s


In [10]:
len(SE18.find_admissible_permutations())

1296

In [11]:
%time adm_SE18 = SE18.find_admissible_projectivities()

CPU times: user 1.35 s, sys: 87.9 ms, total: 1.43 s
Wall time: 12.5 s


In [12]:
len(adm_SE18)

648

In [13]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)

CPU times: user 1.4 s, sys: 168 ms, total: 1.57 s
Wall time: 2.27 s


In [14]:
len(simm_SE18)

648

In [44]:
%%time
perms = []
for el in simm_SE18:
    perms.append(Permutation(SE18.apply_proj_to_eck(el)).to_permutation_group_element())

CPU times: user 12.8 s, sys: 108 ms, total: 12.9 s
Wall time: 12.9 s


In [17]:
gp = PermutationGroup(perms)

In [18]:
gp.order()

648

In [19]:
gp.structure_description()

'(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2'

In [20]:
nor_subs = gp.normal_subgroups()

In [21]:
nor_sub = [el for el in nor_subs if el.order()==27][0]

In [22]:
nor_sub.structure_description()

'C3 x C3 x C3'

In [23]:
S4 = gp.quotient(nor_sub)

In [24]:
S4.is_isomorphic(SymmetricGroup(4))

True

In [45]:
%time sub = gp.subgroups()

CPU times: user 1min 23s, sys: 73.7 ms, total: 1min 23s
Wall time: 1min 23s


In [26]:
len(sub)

928

In [27]:
subS4 = [el for el in sub if el.is_isomorphic(S4)][0]

In [28]:
subS4.gens_small()

[(1,5,9)(2,7,17)(3,13,10)(4,8,12)(6,11,15)(14,18,16),
 (1,2)(3,4,15,16)(5,17,7,9)(6,12,8,10)(11,13,18,14)]

In [29]:
gens_indices = [perms.index(gen) for gen in subS4.minimal_generating_set()]

In [30]:
eck_labels = SE18.eckardt_points_labels
lines_three_eck = []
for i in range(16):
    for j in range(i+1, 17):
        for k in range(j+1, 18):
            point1 = SE18.eckardt_points[i]
            point2 = SE18.eckardt_points[j]
            point3 = SE18.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_planes(line12_pl)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))

In [31]:
len(lines_three_eck)

42

In [32]:
planes = []
for line in lines_three_eck:
    for line2 in line.get_all_lines_incident_to_self(lines_three_eck):
        planes.append(get_plane_containing_two_incident_lines(line, line2))
planes = list(set(planes))

In [33]:
len(planes)

31

In [34]:
c9_planes = [plane for plane in planes if [line.is_on_plane(plane) for line in lines_three_eck].count(True) == 12]

In [35]:
c9_planes

[x + (alpha - 1)*y - t,
 x + (-1/4*alpha - 1/4)*y + (1/4*alpha + 1/4)*z,
 x + (-alpha - 1)*y - t,
 x + (1/4*alpha - 1/4)*y + (-1/4*alpha + 1/4)*z]

In [36]:
simms = [simm_SE18[index] for index in gens_indices]

In [37]:
for simm in simms:
    coord_change = change_coord(simm)
    new_planes = [plane.subs(coord_change) for plane in c9_planes]
    for new_plane in new_planes:
        print(bool([matrix([plane_coefficients(old_plane), plane_coefficients(new_plane)]).minors(2) == [0,0,0,0,0,0] for old_plane in c9_planes]))

True
True
True
True
True
True
True
True


In [39]:
keys = list(SE18.cl_lines.keys())

In [40]:
%%time 
perms_lines = []
for simm in simm_SE18:
    perms_lines.append(Permutation(SE18.apply_proj_to_lines(simm)).to_cycles())

CPU times: user 1min 36s, sys: 992 ms, total: 1min 37s
Wall time: 1min 38s


In [41]:
Gr = PermutationGroup(perms_lines)

In [42]:
Gr.order()

648

In [43]:
Gr.structure_description()

'(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2'